In [ ]:
!pip install transformers
!pip install nlp #restart kernal to use the newest vers. of nlp after installation

In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Loading Model and Tokenizer

*   Refer to https://huggingface.co/transformers/ for Pretrained Models and Tokenizers



In [ ]:
from transformers import ElectraTokenizerFast, ElectraForSequenceClassification

model_name = 'google/electra-base-discriminator'
#model = ElectraForSequenceClassification.from_pretrained(model_name, num_labels=5)
tokenizer = ElectraTokenizerFast.from_pretrained(model_name)

In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased')

Data Preprocessing

In [ ]:
from nlp import load_dataset

train_dataset = load_dataset('csv', data_files='/content/drive/My Drive/Sentiment Analysis/Clean/train1.csv', split='train[:80%]')
val_dataset = load_dataset('csv', data_files='/content/drive/My Drive/Sentiment Analysis/Clean/train1.csv', split='train[-20%:]') #last 20%

In [ ]:
def tokenize(batch):
    return tokenizer(batch['review'],add_special_tokens=True, return_token_type_ids=False, padding=True, truncation=True)

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
val_dataset = val_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

Training the Model

In [ ]:
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

training_args = TrainingArguments(
    output_dir='/content/drive/My Drive/Sentiment Analysis/Checkpoints', # output directory
    num_train_epochs=20,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='/content/drive/My Drive/Sentiment Analysis/logs', # directory for storing logs
    save_steps=10_000,
    logging_steps=10_000,
    eval_steps=10_000,
    evaluate_during_training=True
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,         # training dataset, uses RandomSampler
    eval_dataset=val_dataset           # evaluation dataset, uses SequentialSampler
)

In [ ]:
%%time
trainer.train()

In [ ]:
%%time
trainer.evaluate()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

Saving the Model

In [ ]:
trainer.save_model('/content/drive/My Drive/Sentiment Analysis/Electra')

Loading the Model

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("/content/drive/My Drive/Sentiment Analysis/TPU_Checkpoints/model.bin")

UnicodeDecodeError: ignored

Prediction using Trainer

1.   Load the Model
2.   Do not run the train_dataset again



Load the Test Data

In [ ]:
from nlp import Dataset
import pandas as pd

df = pd.read_csv('/content/drive/My Drive/Sentiment Analysis/Clean/test.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60427 entries, 0 to 60426
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  60427 non-null  object
dtypes: object(1)
memory usage: 472.2+ KB


In [ ]:
df.to_pickle('/content/test.pkl') #save df to a pickle (load_dataset only accept pickle for pandas)

In [ ]:
from nlp import load_dataset

test_dataset = load_dataset('pandas', data_files='/content/test.pkl', split='train[:100%]') #load in 100%

Using custom data configuration default


Dataset pandas downloaded and prepared to /root/.cache/huggingface/datasets/pandas/default/0.0.0. Subsequent calls will reuse this data.


In [ ]:
test_dataset 

Dataset(schema: {'review': 'string'}, num_rows: 60427)

In [ ]:
def tokenize(batch):
    return tokenizer(batch['review'],add_special_tokens=True, return_token_type_ids=False, padding=True, truncation=True)

test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask'])

100%|██████████| 1/1 [00:06<00:00,  6.95s/it]


Trainer for Prediction

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='/content/drive/My Drive/Sentiment Analysis/Checkpoints', # output directory
    per_device_eval_batch_size=32,   # batch size for prediction
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
)

AttributeError: ignored

In [ ]:
results = trainer.predict(test_dataset)

In [ ]:
print (results)

In [ ]:
import numpy as np

labels = [np.argmax(score, axis=-1) for score in results][0] #Select the first array
labels

Export to CSV

In [ ]:
keys = {0: 1, 1: 2, 2: 3, 3: 4, 4: 5}

rating = []
review_id = []

for i in labels:
  rating.append(keys[i])

for y in range(len(labels)):
  review_id.append(y+1)

In [ ]:
import pandas as pd

df_result = pd.DataFrame({'review_id':review_id, 'rating':rating})
df_result.set_index('review_id', inplace=True)

In [ ]:
df_result.to_csv('/content/drive/My Drive/Sentiment Analysis/result.csv')
df_result

Pipeline Prediction

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/My Drive/Sentiment Analysis/clean/test.csv')

reviews = df['review'].tolist()

In [ ]:
%%time
#Run this for batch_size of 1

for i in reviews:
  print (i)
  result = classifier(i)
  results.append(result)

In [ ]:
%%time
batch_size = 16
results = []

i = -(batch_size)
while i < (len(reviews) - batch_size):
  i += batch_size
  print (i)
  print (reviews[i:i+batch_size])
  result = classifier(reviews[i:i+batch_size])
  results.append(result)

In [ ]:
print (results)